# Neighborhoods in Toronto
## Part 1: explore and cluster the neighborhoods in Toronto.

Use [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/) to scrape the [List of postal codes of Canada: M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) page and extract the data from the table into a padnas DataFrame. 

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
data = r.text
soup = BeautifulSoup(data, 'html.parser')
table = soup.table

Preliminary analysis of the dataframe

In [3]:
df = pd.read_html(str(table))[0]
print(df.head())
print(df.shape)

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
(287, 3)


Remove rows where Borough is 'Not Assigned',assign Borough value to Neighbourhood when 'Not Assigned' then join rows with the same Postcode 

In [4]:
df = df[df["Borough"]!="Not assigned"]
df['Neighbourhood'] = np.where(df['Neighbourhood']=='Not assigned',df['Borough'], df['Neighbourhood'])
df=df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [5]:
print(df.shape)

(103, 3)


# Neighborhoods in Toronto
## Part 2: Use Geocoder or the csv to extract Latitude and Longitude from postal codes

In [6]:
dfPC=pd.read_csv("https://cocl.us/Geospatial_data")
df["Latitude"] = 0
df["Longitude"] = 0
for postcode in df["Postcode"]:
    lat_lng_coords=dfPC[dfPC["Postal Code"]==postcode][["Latitude","Longitude"]]
    df.at[df["Postcode"]==postcode,"Latitude"]=lat_lng_coords.iloc[0,0]
    df.at[df["Postcode"]==postcode,"Longitude"]=lat_lng_coords.iloc[0,1]

In [8]:
print(df)
print(df.shape)

    Postcode      Borough                                      Neighbourhood  \
0        M1B  Scarborough                                      Rouge,Malvern   
1        M1C  Scarborough               Highland Creek,Rouge Hill,Port Union   
2        M1E  Scarborough                    Guildwood,Morningside,West Hill   
3        M1G  Scarborough                                             Woburn   
4        M1H  Scarborough                                          Cedarbrae   
..       ...          ...                                                ...   
98       M9N         York                                             Weston   
99       M9P    Etobicoke                                          Westmount   
100      M9R    Etobicoke  Kingsview Village,Martin Grove Gardens,Richvie...   
101      M9V    Etobicoke  Albion Gardens,Beaumond Heights,Humbergate,Jam...   
102      M9W    Etobicoke                                          Northwest   

      Latitude  Longitude  
0    43.806